In [1]:
import numpy as np
import re
from openbabel import openbabel
from openbabel import pybel
import os
import shutil
import math

hartree2kcalmol = 627.5094740631
T = 298.15 
k_Boltzmann = 0.001987204259

In [2]:
def vecBO3_complex(mol,idxs):
    B_idx = 0
    for idx in idxs:
        dummy_atom = mol.OBMol.GetAtom(idx)
        if dummy_atom.GetAtomicNum() == 8:
            for atom in openbabel.OBAtomAtomIter(dummy_atom):
                if atom.GetAtomicNum() == 5:
                    #if B_idx != atom.GetIdx():
                    #    B_idx = atom.GetIdx()
                    Bvec = atom.GetVector()
                    #print("Boron atom found",atom.GetIdx())
                    Netvec = [0, 0, 0]
                    for neigh_atom in openbabel.OBAtomAtomIter(atom):
                        if neigh_atom.GetAtomicNum() == 8:
                            #print("Oxigen bonded to Boron is found",neigh_atom.GetIdx())
                            Ovec = neigh_atom.GetVector()
                            BOvec = [Ovec.GetX()-Bvec.GetX(), \
                                     Ovec.GetY()-Bvec.GetY(), \
                                     Ovec.GetZ()-Bvec.GetZ()]
                            vector = openbabel.vector3()
                            vector.Set(BOvec[0],BOvec[1],BOvec[2])
                            vector.normalize() 
                            Netvec[0] += vector.GetX()
                            Netvec[1] += vector.GetY()
                            Netvec[2] += vector.GetZ()
                            #print(Netvec, vector.length(),vector.GetX(),",",vector.GetY(),",",vector.GetZ())
                    vector = openbabel.vector3()
                    vector.Set(Netvec[0],Netvec[1],Netvec[2])
                    vector.normalize()
                    #print(Netvec, vector.length(),vector.GetX(),",",vector.GetY(),",",vector.GetZ())
                    return vector.GetX(),vector.GetY(),vector.GetZ(), atom.GetIdx()

def vecBO3_freeligand(mol,idx):
    B_atom = mol.OBMol.GetAtom(idx)
    #print("Atomic Num atom",B_atom.GetAtomicNum(),B_atom.GetIdx())
    Bvec = B_atom.GetVector()                
    Netvec = [0, 0, 0]
    for neigh_atom in openbabel.OBAtomAtomIter(B_atom):
        if neigh_atom.GetAtomicNum() == 8:
            #print("Oxigen bonded to Boron is found",neigh_atom.GetIdx())
            Ovec = neigh_atom.GetVector()
            BOvec = [Ovec.GetX()-Bvec.GetX(), \
                     Ovec.GetY()-Bvec.GetY(), \
                     Ovec.GetZ()-Bvec.GetZ()]
            vector = openbabel.vector3()
            vector.Set(BOvec[0],BOvec[1],BOvec[2])
            vector.normalize() 
            Netvec[0] += vector.GetX()
            Netvec[1] += vector.GetY()
            Netvec[2] += vector.GetZ()
            #print(Netvec, vector.length(),vector.GetX(),",",vector.GetY(),",",vector.GetZ())
    vector = openbabel.vector3()
    vector.Set(Netvec[0],Netvec[1],Netvec[2])
    vector.normalize()
    #print(Netvec, vector.length(),vector.GetX(),",",vector.GetY(),",",vector.GetZ())
    return vector.GetX(),vector.GetY(),vector.GetZ()

def Boltzmann_avg(G_min,G_mols,BB_dist,vecs_glcyl,vecs_frcyl):
    Z = 0
    z = []

    for g in G_mols:
        G_mol = (g - G_min)*hartree2kcalmol
        Z += math.exp(-G_mol/(k_Boltzmann*T))
        z.append(math.exp(-G_mol/(k*T)))

    B_dist_avg = 0
    Vec_glcyl_avg = [0,0,0]
    Vec_frcyl_avg = [0,0,0]
    for g,d,vec_glcyl,vec_frcyl in zip(G_mols,BB_dist,vecs_glcyl,vecs_frcyl):
        G_mol = (g - G_min)*hartree2kcalmol
        p = math.exp(-G_mol/(k_Boltzmann*T))/Z
        B_dist_avg += p * d 
        for i in range(len(Vec_glcyl_avg)):
            Vec_glcyl_avg[i] += p * vec_glcyl[i]
        for i in range(len(Vec_frcyl_avg)):
            Vec_frcyl_avg[i] += p * vec_frcyl[i]
            
    return B_dist_avg, Vec_glcyl_avg, Vec_frcyl_avg

In [7]:
G_min = 100000
G_mols = []
frcyl_smarts = pybel.Smarts('C1(O)C(O)C(CO)OC1(CO)') 
glcyl_smarts = pybel.Smarts('C1C(O)C(O)C(CO)OC1(O)')
#linker_smarts = pybel.Smarts('[98*]C1=CC=CC2=C1CC1=CC3=C(C=C1C2)CCC=C3C1=CC([99*])=CC=C1')
linker_smarts = pybel.Smarts('[98*]c1cccc2c1Cc1cc3c(cc1C2)CCC=C3c1cccc([99*])c1')
#linker_mol = pybel.readstring("smi","[98*]C1=CC=CC2=C1CC1=CC3=C(C=C1C2)CCC=C3C1=CC([99*])=CC=C1")
linker_mol = pybel.readstring("smi","[98*]c1cccc2c1Cc1cc3c(cc1C2)CCC=C3c1cccc([99*])c1")
vecs_glcyl = []
vecs_frcyl = []
BB_dist = []

for mol in pybel.readfile("xyz", "enso_ensemble_part1.xyz"):
    #print(mol.OBMol.GetMolWt(),mol.OBMol.GetTitle())
    title = mol.OBMol.GetTitle()
    g = float(title.split()[1])
    G_mols.append(g)
    if g < G_min:
        G_min = g

    frcyl_smarts.obsmarts.Match(mol.OBMol)
    glcyl_smarts.obsmarts.Match(mol.OBMol)

    #print("Matches:", glcyl.findall(mol),frcyl.findall(mol))
    #Here write an if conditional to decide if I should use the complex or the 
    #ligand routine to get the vectors and distances.
    glcyl_idxs = glcyl_smarts.findall(mol)[0]
    frcyl_idxs = frcyl_smarts.findall(mol)[0]

    x, y, z, B_idx_glcyl = vecBO3_complex(mol,glcyl_idxs)
    vecs_glcyl.append([x,y,z])
    
    x, y, z, B_idx_frcyl = vecBO3_complex(mol,frcyl_idxs)
    vecs_frcyl.append([x,y,z])

    B_atom_glcyl = mol.OBMol.GetAtom(B_idx_glcyl)
    B_atom_frcyl = mol.OBMol.GetAtom(B_idx_frcyl)
    BB_dist.append(B_atom_glcyl.GetDistance(B_atom_frcyl))

#print(vecs_glcyl,"\n", vecs_frcyl,"\n", BB_dist)

print(Boltzmann_avg(G_min,G_mols,BB_dist,vecs_glcyl,vecs_frcyl))

(7.886773087553523, [0.9642681475112322, -0.16502060072453437, -0.2063285008005871], [0.8741877089882266, -0.24969694461963798, 0.4158292471116788])


In [8]:
G_min = 10000
G_mols = []
vecs_glcyl = []
vecs_frcyl = []
BB_dist = []

for mol in pybel.readfile("xyz", "../../free_ligand/results_censo_job_71801/enso_ensemble_part1.xyz"):
    #print(mol)
    linker_mol = pybel.readstring("smi","[98*]c1cccc2c1Cc1cc3c(cc1C2)CCC=C3c1cccc([99*])c1")
    title = mol.OBMol.GetTitle()
    g = float(title.split()[1])
    G_mols.append(g)
    if g < G_min:
        G_min = g
    
    for atom in openbabel.OBMolAtomIter(linker_mol.OBMol):
        if atom.GetAtomicNum() == 0 and atom.GetIsotope()== 98:
            B_glcyl = atom.GetIdx()
            #print("Dummy atom glcyl side",atom.GetIsotope())
            #for neigh_atom in openbabel.OBAtomAtomIter(atom):
                #C_glcyl = neigh_atom.GetIdx()
                #print("Carbon atom bonded to Dummy in glcyl side",neigh_atom.GetIdx())
        elif atom.GetAtomicNum() == 0 and atom.GetIsotope()== 99:
            B_frcyl = atom.GetIdx()
            #print("Dummy atom frcyl side",atom.GetIsotope())
            #for neigh_atom in openbabel.OBAtomAtomIter(atom):
                #C_frcyl = neigh_atom.GetIdx()
                #print("Carbon atom bonded to Dummy in frcyl side",neigh_atom.GetIdx())
    
    rxn98 = openbabel.OBChemTsfm()
    rxn99 = openbabel.OBChemTsfm()
    rxn98.Init("[98*]","B")
    rxn99.Init("[99*]","B")
    rxn98.Apply(linker_mol.OBMol)
    rxn99.Apply(linker_mol.OBMol)
    smiles = linker_mol.write("smi")
    smiles = smiles.replace('98','')
    smiles = smiles.replace('99','')
    #print(smiles)
    linker_smarts = pybel.Smarts(smiles)
    linker_smarts.obsmarts.Match(mol.OBMol)
    matches = linker_smarts.findall(mol)[0]
    #print("Matches:", linker_smarts.findall(mol)[0])
    #print("B glcyl side:", matches[B_glcyl-1])
    #print("B frcyl side:", matches[B_frcyl-1])
    B_glcyl_idx = matches[B_glcyl-1]
    B_frcyl_idx = matches[B_frcyl-1]

    x, y, z = vecBO3_freeligand(mol,B_glcyl_idx)
    vecs_glcyl.append([x,y,z])
    #print("New side")
    x, y, z = vecBO3_freeligand(mol,B_frcyl_idx)
    vecs_frcyl.append([x,y,z])

    B_atom_glcyl = mol.OBMol.GetAtom(B_idx_glcyl)
    B_atom_frcyl = mol.OBMol.GetAtom(B_idx_frcyl)
    BB_dist.append(B_atom_glcyl.GetDistance(B_atom_frcyl))

print(Boltzmann_avg(G_min,G_mols,BB_dist,vecs_glcyl,vecs_frcyl))

(6.245520284185368, [0.5702089631882846, -0.7562392059617652, -0.24611481276595612], [-0.5101442965732257, -0.7904564914154943, 0.32864955026374465])
